In [ ]:
#!conda list
####!conda install pytorch torchvision -c pytorch -y
#!conda install ipython -y
#!python -m pip install torch

# Load bespin-global/klue-bert-base-mrc
#!pip install ipywidgets
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
print("로딩 중(~5초)")
#print(torch.__version__)

#infile = "papersubq.tsv"
#outfile = "3paper_bespin.tsv"
infile = "papersubq-word.tsv"
outfile = "3paper-word_bespin.tsv"

# Load fine-tuned MRC model by HuggingFace Model Hub
HUGGINGFACE_MODEL_PATH = "bespin-global/klue-bert-base-mrc"
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH )
model = AutoModelForQuestionAnswering.from_pretrained(HUGGINGFACE_MODEL_PATH )

def bespin(context, question):
    # Encoding
    encodings = tokenizer(context, question, 
                          max_length=512, 
                          truncation=True,
                          padding="max_length", 
                          return_token_type_ids=False
                          )
    encodings = {key: torch.tensor([val]) for key, val in encodings.items()}             
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]

    # Predict
    pred = model(input_ids, attention_mask=attention_mask)

    start_logits, end_logits = pred.start_logits, pred.end_logits
    token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
    pred_ids = input_ids[0][token_start_index: token_end_index + 1]

    # Decoding
    prediction = tokenizer.decode(pred_ids)
    return prediction

inf = open(infile);
inf.readline();
lines = inf.readlines();
inf.close();
out = open(outfile,"w");


i = 0;
#out.write("서브문제\t정답\t지문\t콤마파트\t문장전체\t문장고르기\n");
for line in lines:
    i = i+1;
    if(i%100==0):
        print(str(i));
    t = line.split("\t");
    question = t[0].replace("|","");
    answer = t[1].strip();
    context = t[2].replace("|","");
    sent = t[4];
    regex = t[5];
    comma = ""
    if(len(t)>6):
        comma = t[6].strip();
    
    pred = bespin(context,question);
    if(pred.find('[SEP]')>0):
        pred = pred[0:pred.find('[SEP]')]
    
    out.write(question+"\t"+answer+"\t"+pred+"\t"+context+"\t"+sent+"\t"+regex+"\t"+comma+"\n");
    
    #if(i%30==0):
    if(i%10==0):
        print(comma+"\t"+answer+"\t"+pred+"\t"+question+"\n");
out.close();

context = "연구대상자는 17세 이하 503 명 "
question = "연구대상자는연구대상자는 17세 이하  명 ?"
bespin(context,question)
print(bespin(context,question))


로딩 중(~5초)
1	임상진료지침의 개발기관	임상진료지침	이 사이트에서는 검색된  다양한 형태로 원문을 제공하고 있다?

2	개발 연도	개발 연도	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

3	범위	이 사이트	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

4	방법론	이 사이트에서는 검색된 방법론 다양한 형태로 원문을 제공하고 있다. 	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

5	권고내용 등의 간략정보를 제공할 뿐 아니라 Complete Summary	Complete Summary	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

6	XML View	XML View	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

7	Full Text	Full Text	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

8	Palm Download	Palm Download	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

9	MS Word	MS Word	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

10	PDF 등의	PDF 등의 다양한 형태로 원문을 제공하고 있다. 	이 사이트에서는 검색된 다양한 형태로 원문을 제공하고 있다?

1	predictation	predictation	진단 논문에 많이 나타나는 단어들은  등으로 이런 단어들을 어떻게 조합하여 검색했을 경우 특이도나 민감도 높은 검색을 할 수 있는지 연구하는 것이다?

2	diagnosis	diagnosis	진단 논문에 많이 나타나는 단어들은 등으로 이런 단어들을 어떻게 조합하여 검색했을 경우 특이도나 민감도 높은 검색을 할 수 있는지 연구하는 것이다?

3	sensitivity	sensitivity	진단 논문에 많이 나타나는 단어들은 등으로 이런 단어들을 어떻게 조합하여 검색했을 경우 특이도나 민감도 높은 검색을 할 수 있는지 연구하는 것이다?

4	specificity	specificity	진단 